## 4.1

In [1]:
function make_H(N,L,V)
    Δx=L/(N+1)
    H=zeros(Float64,N,N)
    for i=1:N
        x=i*Δx
        H[i,i]=V(x)

        j=i+1
        dij=-1/Δx^2
        if 1≤j≤N
            H[i,j]+=dij
        end
        
        j=i
        dij=2/Δx^2
        if 1≤j≤N
            H[i,j]+=dij
        end

        j=i-1
        dij=-1/Δx^2
        if 1≤j≤N
            H[i,j]+=dij
        end
    end
    return H
end

make_H (generic function with 1 method)

In [2]:
using LinearAlgebra

In [4]:
using Plots

In [6]:
function test()
    V(x)=0
    N=1000
    L=1
    H=make_H(N,L,V)
    e,v=eigen(H)
    e0=zeros(Float64,N)
    for n=1:N
        e0[n]=n^2*π^2/L^2
    end
    plot(1:N,[e,e0],
        labels=["Numerical result" "Analytical result"],
        xlabel="n",
        ylabel="energy")
        savefig("eigen.png")
        println(e0[1],"\t",e[1])
end
test()

9.869604401089358	9.869596300201396


In [11]:
function test2()
    V(x) = 0
    N = 1000
    L = 1
    H = make_H(N,L,V)
    e,v = eigen(H)

    Δx = L/(N+1)
    xs = zeros(Float64,N)
    ψ0 = zeros(Float64,N)
    ψ250 = similar(ψ0)
    n = 1
    m = 250
    for i=1:N
        x = i*Δx
        xs[i] = x
        ψ0[i] = sqrt(2/L)*sin(x*n*π/L)
        ψ250[i] = sqrt(2/L)*sin(x*m*π/L)
    end
    coeff = 1/sqrt(Δx)

    plot(xs,coeff*v[:,n],label="Numerical result n=1",xlabel="x",ylabel="psi(x)")
    plot!(xs,coeff*v[:,m],label="Numerical result n=250",xlabel="x",ylabel="psi(x)")
    plot!(xs,ψ0,label="Analytical result n=1",xlabel="x",ylabel="psi(x)")
    plot!(xs,ψ250,label="Analytical result n=250",xlabel="x",ylabel="psi(x)")
    savefig("psi.png")
end
test2()

In [10]:
function test2()
    V(x) = 0
    N = 1000
    L = 1
    H = make_H(N,L,V)
    e,v = eigen(H)
    
    Δx = L/(N+1)
    xs = zeros(Float64,N)
    ψ0 = zeros(Float64,N)
    ψ250 = similar(ψ0)
    n = 1
    m = 250
    for i=1:N
        x = i*Δx
        xs[i] = x
        ψ0[i] = sqrt(2/L)*sin(x*n*π/L)
        ψ250[i] = sqrt(2/L)*sin(x*m*π/L)
    end
    coeff = 1/sqrt(Δx)
    
    plot(xs,coeff*v[:,n],label="Numerical result n=1",xlabel="x",ylabel="psi(x)")
    plot!(xs,ψ0,label="Analytical result n=1",xlabel="x",ylabel="psi(x)")
    plot!(xs,coeff*v[:,m],label="Numerical result n=250",xlabel="x",ylabel="psi(x)")
    plot!(xs,ψ250,label="Analytical result n=250",xlabel="x",ylabel="psi(x)")
    savefig("psi.png")
end
test2()

In [12]:
function calc_vq(q,ξ,V0)
    vq=sqrt(π*ξ^2)*exp(-q^2*ξ^2/4)
    return vq
end

calc_vq (generic function with 1 method)

In [19]:
function calc_Vkkp(k,kp,L,ξ,x0,V0)
    q1=k-kp
    vq1=calc_vq(q1,ξ,V0)
    q2=k+kp
    vq2=calc_vq(q2,ξ,V0)
    Vkkp=(V0/L)*(cos(q1*x0)*vq1-cos(q2*x0)*vq2)
    return Vkkp
end

calc_Vkkp (generic function with 1 method)

In [20]:
function make_Hk(N,L,ξ,x0,V0)
    mat_Hk = zeros(Float64,N,N)
    for n in 1:N
        k = n*π/L
        for np in 1:N
            if n == np
                v = k^2
            else
                v = 0
            end
            kp = np*π/L
            Vkkp = calc_Vkkp(k,kp,L,ξ,x0,V0)
            v += Vkkp
            mat_Hk[n,np] = v
        end
    end
    return mat_Hk
end

make_Hk (generic function with 1 method)

In [21]:
function calc_psi(cn,x,L)
    nmax = length(cn)
    psi = 0
    for n=1:nmax
        kn = n*π/L
        psi += cn[n]*sin(kn*x)
    end
    return psi*sqrt(2/L)
end

calc_psi (generic function with 1 method)

In [22]:
function momentumspace(N,L,ξ,x0,V0)
    Hk = make_Hk(N,L,ξ,x0,V0)
    ep,bn = eigen(Hk)
    xs = range(0,L,length=N)
    psi = zeros(Float64,N)
    n = 1
    for (i,x) in enumerate(xs)
        psi[i]  =calc_psi(bn[:,n],x,L)
    end
    plot(xs,psi,label="Numerical result in momentum space: n=1", xlabel="x",ylabel="psi(x)")
    savefig("momu.pdf")
    return
end

momentumspace (generic function with 1 method)

In [23]:
N = 1000
L = 10
ξ = 1
x0 = L/2
V0 = 1

momentumspace(N,L,ξ,x0,V0)

In [24]:
function gaussV(x,ξ,x0,V0)
    V0*exp(-(x-x0)^2/ξ^2)
end

gaussV (generic function with 1 method)

In [26]:
V(x) = gaussV(x,ξ,x0,V0)
Hreal = make_H(N,L,V)
e,v = eigen(Hreal)
println(e[1:10])
Hk =  make_Hk(N,L,ξ,x0,V0)
e,v = eigen(Hk)
println(e[1:10])

[0.3318732019153381, 0.4445184018941452, 1.1891596002955234, 1.711703103024328, 2.6885089824996653, 3.7252411323786125, 5.023736585633207, 6.495898312608702, 8.174847015417331, 10.048398020833162]
[0.3318740815369017, 0.44451985491236623, 1.1891669770624456, 1.7117244036279688, 2.688559578455572, 3.7253467453719855, 5.023931236271939, 6.496230600508955, 8.175378836500212, 10.049208513174639]
